In [1]:
import pandas as pd
import pandas

import matplotlib.pyplot as plt

import os

import numpy as np
import numpy
np.random.seed(42)

import seaborn as sns
import seaborn

import sklearn
import sklearn as sk

import copy
from collections import OrderedDict
%matplotlib inline

import seaborn
import seaborn as sns

In [2]:
data_classes = pd.read_csv('data/data7.csv')
data_static = pd.read_csv('raw_data/Static.csv', ';').drop('CLIENT_ID', axis=1)
data_join = pd.merge(data_static, data_classes, on='CONTRACT_ID')

In [3]:
contracts = np.array(data_static['CONTRACT_ID'].tolist())
shuffled  = copy.deepcopy(contracts)

length    = contracts.shape[0]
RATIO     = 0.1

test_contracts  = shuffled[:int(length * RATIO)].tolist()
train_contracts = shuffled[int(length * RATIO):].tolist()

data_learn = data_join.dropna(axis=0)

data_learn['GENDER'] = 1 * np.array(data_learn['GENDER'] == 'M')

data_learn['RATIO_10'] = 1 * np.array(data_learn['LOAN_TO_INCOME'] >= 10)
data_learn['RATIO_20'] = 1 * np.array(data_learn['LOAN_TO_INCOME'] >= 20)
data_learn['RATIO_30'] = 1 * np.array(data_learn['LOAN_TO_INCOME'] >= 30)
data_learn['RATIO_40'] = 1 * np.array(data_learn['LOAN_TO_INCOME'] >= 40)
data_learn['BEFORE_GRACE']   = data_learn['PAYMENT_TO_INCOME'] / data_learn['LOAN_TO_INCOME'] * 100

data_train = data_learn[data_learn['CONTRACT_ID'].isin(train_contracts)].drop('CONTRACT_ID', axis=1)
data_test  = data_learn[data_learn['CONTRACT_ID'].isin( test_contracts)].drop('CONTRACT_ID', axis=1)

In [4]:
data_train.columns

Index(['TERM', 'CONTRACT_SUM', 'GENDER', 'AGE', 'LOAN_TO_INCOME',
       'PAYMENT_TO_INCOME', 'DOWNPAYMENT', 'CAR_CATEGORY', 'GRACE_PERIOD',
       'RATE_CHANGE_AFTER_GRACE', 'km_8', 'km_16', 'RATIO_10', 'RATIO_20',
       'RATIO_30', 'RATIO_40', 'BEFORE_GRACE'],
      dtype='object')

In [5]:
# ways = ['sdtw_km', 'dba_km', 'km', 'gak_km', 'ks']
ways = ['km_8', 'km_16']
data_feats = {
    'train' : data_train.drop(columns=ways),
    'test'  :  data_test.drop(columns=ways)
}
data_classes = {
    'train' : {},
    'test'  : {}
}
for item in ways:
    data_classes['train'][item] = data_train[item]
    data_classes['test'][item] = data_test[item]

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score

from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators' : [100, 300, 500, 1000],
    'max_depth' : [5, 10, 20, 40, 50, 100],
    'min_samples_leaf' : [1, 2, 5, 10],
    'random_state' : [42]
}

rf_algos = {}
rf_scores = {}
rf_predictions = {}

for way in ways:
    clf_rf = GridSearchCV(RandomForestClassifier(), param_grid)
    clf_rf = RandomForestClassifier(n_estimators=500, max_depth=40, random_state=42, min_samples_leaf=5)
    clf_rf.fit(data_feats['train'], data_classes['train'][way])
    rf_predictions[way] = clf_rf.predict(data_feats['test'])
    rf_algos[way]  = clf_rf
    rf_scores[way] = {
        'accuracy' : accuracy_score(data_classes['test'][way], rf_predictions[way]),
        'f1_score' : f1_score(data_classes['test'][way], rf_predictions[way], average='weighted')
    }

/Users/work/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/work/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/work/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/work/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [18]:
for key, report in rf_scores.items():
    print(key, '\n', report)

km_8 
 {'accuracy': 0.41424802110817943, 'f1_score': 0.38814393755368637}
km_16 
 {'accuracy': 0.22163588390501318, 'f1_score': 0.14128092872116146}


In [8]:
import pickle
pickle.dump(rf_algos, open('pickle_data/rf_clf_7.pickle', 'wb'))

In [9]:
import xgboost

xgb_algos = {}
xgb_scores = {}
xgb_predictions = {}

param_grid = {
    'booster' :      ['dart', 'gbtree'],
    'max_depth' :    [6, 10, 20, 30, 40],
    'lambda' :       [1, 10, 0.1],
    'random_state' : [42]
}


for way in ways[0]:
    print("Processing way: ", way)
    xgb_clf = GridSearchCV(xgboost.XGBClassifier(), param_grid)
    xgb_clf.fit(data_feats['train'], data_classes['train'][way])
    xgb_predictions[way] = xgb_clf.predict(data_feats['test'])
    xgb_algos[way]  = xgb_clf
    xgb_scores[way] = {
        'accuracy' : accuracy_score(data_classes['test'][way], xgb_predictions[way]),
        'f1_score' : f1_score(data_classes['test'][way], xgb_predictions[way], average='weighted')
    }
    


Processing way:  km_8


/Users/work/Library/Python/3.7/lib/python/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/work/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/work/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/work/Library/Python/3.7/lib/python/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, 

Processing way:  km_16


KeyboardInterrupt: 

In [10]:
for key, report in xgb_scores.items():
    print(key, '\n', report)

km_8 
 {'accuracy': 0.46701846965699206, 'f1_score': 0.4290268665763388}


In [19]:
pickle.dump(xgb_algos, open('pickle_data/xgb_clf_7.pickle', 'wb'))